In [10]:
import sqlite3

# Function to create tables
def create_tables():
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()

    # Create Books table
    cursor.execute('''
        CREATE TABLE Books (
            book_id INTEGER PRIMARY KEY,
            title TEXT,
            author TEXT,
            genre TEXT
        )
    ''')

    # Create Members table
    cursor.execute('''
        CREATE TABLE Members (
            member_id INTEGER PRIMARY KEY,
            name TEXT,
            email TEXT,
            phone TEXT
        )
    ''')

    # Create Borrowing table
    cursor.execute('''
        CREATE TABLE Borrowing (
            borrowing_id INTEGER PRIMARY KEY,
            book_id INTEGER,
            member_id INTEGER,
            borrow_date DATE,
            return_date DATE,
            FOREIGN KEY (book_id) REFERENCES Books (book_id),
            FOREIGN KEY (member_id) REFERENCES Members (member_id)
        )
    ''')

    conn.commit()
    conn.close()

    print("Tables created successfully.")

# Function to insert sample data
def insert_sample_data():
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()

    # Insert a new book
    cursor.execute('''
        INSERT INTO Books (title, author, genre)
        VALUES ('Python Programming', 'John Doe', 'Programming')
    ''')

    # Insert a new member
    cursor.execute('''
        INSERT INTO Members (name, email, phone)
        VALUES ('Alice Smith', 'alice@example.com', '123-456-7890')
    ''')

    # Insert a new borrowing
    cursor.execute('''
        INSERT INTO Borrowing (book_id, member_id, borrow_date, return_date)
        VALUES (1, 1, '2024-07-02', NULL)
    ''')

    conn.commit()
    conn.close()

    print("Data inserted successfully.")

# Function to perform SQL injection attack
def perform_sql_injection_attack():
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()

    # Simulate user input (this could potentially be an attacker's input)
    username = "fakeuser' OR '1'='1' --"

    # Vulnerable SQL query (without any security measures)
    sql = f"SELECT * FROM Members WHERE name = '{username}'"

    cursor.execute(sql)
    results = cursor.fetchall()

    print("SQL Injection Results:")
    for row in results:
        print(row)

    conn.close()

# Main function to demonstrate all steps
def main():
    create_tables()
    insert_sample_data()
    
    print("\nPerforming SQL Injection Attack:")
    perform_sql_injection_attack()

if __name__ == "__main__":
    main()



Tables created successfully.


OperationalError: no such table: Books

OperationalError: no such table: Members